# Deployment

# Export Data to Training Pipelines

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/content/diamonds.csv')

In [3]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
# Split Dataset menjadi train, test, dan validation
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [5]:
# Membuat target dari prediksi model

# price 0 = low; 1 = high
df['target'] = np.where(df['price']==327, 0, 1)

# Drop un-used columns.
df = df.drop(columns=['price'])

In [6]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
batch_size = 10 # batch ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [8]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [9]:
# numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[1.53]
 [2.39]
 [0.55]
 [2.  ]
 [1.5 ]
 [0.4 ]
 [1.06]
 [0.4 ]
 [0.71]
 [1.5 ]]


In [10]:
# bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [11]:
# categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])
 
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]]


In [12]:
# Pilih feature column mana yang akan digunakan
feature_columns = []

In [13]:
# embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[ 0.43725246  0.5589956  -0.08827138 -0.4275147  -0.04181847  0.2202511 ]
 [-0.6001344  -0.61763555 -0.15075025  0.3868759  -0.58214444 -0.017605  ]
 [-0.0151603   0.0718402  -0.41967463 -0.20125318  0.19511366 -0.7458131 ]
 [-0.27077296 -0.4337373   0.35020268  0.2772842   0.57942647  0.40740412]
 [-0.27077296 -0.4337373   0.35020268  0.2772842   0.57942647  0.40740412]
 [-0.6001344  -0.61763555 -0.15075025  0.3868759  -0.58214444 -0.017605  ]
 [-0.0151603   0.0718402  -0.41967463 -0.20125318  0.19511366 -0.7458131 ]
 [-0.10820406  0.12765808 -0.08518174  0.27871144 -0.43099025 -0.32373244]
 [-0.27077296 -0.4337373   0.35020268  0.2772842   0.57942647  0.40740412]
 [-0.27077296 -0.4337373   0.35020268  0.2772842   0.57942647  0.40740412]]


In [14]:
# hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [15]:
# cross feature
# data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [16]:
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

In [17]:
# membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [18]:
# create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1685/1686 [============================>.] - ETA: 0s - loss: 4.0137e-04 - accuracy: 1.0000WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Unnamed: 0': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'carat': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'cut': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=string>, 'color': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'clarity': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'depth': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'table': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=float64>, 'x': <tf.Tensor 'ExpandDims_7:0' shape=(None, 1) dtype=float64>, 'y': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=float64>, 'z': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) d

In [19]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1686/1686 [==============================] - 3s 2ms/step - loss: 3.0739e-04 - accuracy: 1.0000
Accuracy 0.9999814629554749
